put the new lncRNA into some categories


In [109]:
import pandas as pd
import seaborn as sns
import pyranges as pr
import numpy as np
import matplotlib.pyplot as plt
import upsetplot as up
from matplotlib.colors import hex2color
from scipy.stats.kde import gaussian_kde
from matplotlib.colors import LogNorm
from scipy import stats
import os
import matplotlib.patches as patches
from statsmodels.stats.proportion import proportion_confint

from tqdm.notebook import tqdm

tqdm.pandas()

/local/scratch/klawren/slrmtmp.48860077/ipykernel_12178/4237669646.py:8: DeprecationWarning: Please import `gaussian_kde` from the `scipy.stats` namespace; the `scipy.stats.kde` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.stats.kde import gaussian_kde


In [ ]:
full_gencode_v47_path = "/home/klawren/oak/gtex/data/other_references/gencode/gencode.v47.annotation.gtf"
full_gencode_v47=pr.read_gtf(full_gencode_v47_path).as_df()



In [159]:
# bring in gencode ids
gencode_ids = pd.read_table('/home/klawren/oak/gtex/data/other_references/gencode/gencode_id_matching.txt')
lncrnas = gencode_ids[gencode_ids['gene_type_grouped_highlight'].isin(['existing_lncRNA', 'new_lncRNA'])]

/local/scratch/klawren/slrmtmp.48860077/ipykernel_12178/3742419282.py:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  gencode_ids = pd.read_table('/home/klawren/oak/gtex/data/other_references/gencode/gencode_id_matching.txt')


In [120]:
gencode_genes = full_gencode_v47[full_gencode_v47['Feature'] == 'gene']
gencode_genes['Strand'].isna().sum()

np.int64(0)

In [170]:
# Annotate lncRNA genes as either 'Antisense' (overlapping a protein-coding gene on the opposite strand)
# or 'Intergenic' (not overlapping any protein-coding gene at all)

# Prepare Gencode v47 annotation DataFrame for gene-level features
gencode_genes = full_gencode_v47[full_gencode_v47['Feature'] == 'gene']
gencode_genes = gencode_genes[['Chromosome', 'Start', 'End', 'Strand', 'gene_id', 'gene_type', 'gene_name']]

# Subset protein-coding genes and lncRNAs
pc_genes = gencode_genes[gencode_genes['gene_type'] == 'protein_coding'].copy()
lnc_genes = gencode_genes[gencode_genes['gene_type'].str.contains('lncRNA', case=False, na=False)].copy()

# Convert to PyRanges for efficient overlap operations
pc_pr = pr.PyRanges(pc_genes)
lnc_pr = pr.PyRanges(lnc_genes)

# Find lncRNAs that overlap a protein-coding gene on the opposite strand (Antisense)
antisense_ids = set()
for strand in ['+', '-']:
    lnc_on_strand = lnc_pr[lnc_pr.Strand == strand]
    pc_on_opposite = pc_pr[pc_pr.Strand == ('-' if strand == '+' else '+')]
    overlap = lnc_on_strand.overlap(pc_on_opposite)
    antisense_ids.update(overlap.df['gene_id'])

# Find lncRNAs that overlap a protein-coding gene on the opposite strand (Antisense)
antisense_ids = set()
for strand in ['+', '-']:
    lnc_on_strand = lnc_pr[lnc_pr.Strand == strand]
    pc_on_opposite = pc_pr[pc_pr.Strand == ('-' if strand == '+' else '+')]
    overlap = lnc_on_strand.overlap(pc_on_opposite)
    antisense_ids.update(overlap.df['gene_id'])

# Find lncRNAs that overlap a protein-coding gene on the same strand (Sense)
sense_ids = set()
for strand in ['+', '-']:
    lnc_on_strand = lnc_pr[lnc_pr.Strand == strand]
    pc_on_same = pc_pr[pc_pr.Strand == strand]
    overlap = lnc_on_strand.overlap(pc_on_same)
    sense_ids.update(overlap.df['gene_id'])

# Find lncRNAs that do not overlap any protein-coding gene (Intergenic)
overlap_any = lnc_pr.overlap(pc_pr)
intergenic_ids = set(lnc_pr.df['gene_id']) - set(overlap_any.df['gene_id'])

# Assign categories
lncrnas['lncrna_category'] = 'other'
lncrnas.loc[lncrnas['gene_id_v47'].isin(antisense_ids), 'lncrna_category'] = 'antisense'
lncrnas.loc[lncrnas['gene_id_v47'].isin(sense_ids), 'lncrna_category'] = 'sense'
lncrnas.loc[lncrnas['gene_id_v47'].isin(antisense_ids) & lncrnas['gene_id_v47'].isin(sense_ids), 'lncrna_category'] = 'overlapping_both_strands'
lncrnas.loc[lncrnas['gene_id_v47'].isin(intergenic_ids), 'lncrna_category'] = 'intergenic'

# Check for overlaps between categories
antisense_set = set(antisense_ids)
sense_set = set(sense_ids)
intergenic_set = set(intergenic_ids)

overlap_antisense_sense = antisense_set & sense_set
overlap_antisense_intergenic = antisense_set & intergenic_set
overlap_sense_intergenic = sense_set & intergenic_set

print(f"Number of lncRNAs in both Antisense and Sense: {len(overlap_antisense_sense)}")
print(f"Number of lncRNAs in both Antisense and Intergenic: {len(overlap_antisense_intergenic)}")
print(f"Number of lncRNAs in both Sense and Intergenic: {len(overlap_sense_intergenic)}")


Number of lncRNAs in both Antisense and Sense: 531
Number of lncRNAs in both Antisense and Intergenic: 0
Number of lncRNAs in both Sense and Intergenic: 0


/local/scratch/klawren/slrmtmp.48860077/ipykernel_12178/1484911030.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lncrnas['lncrna_category'] = 'other'


In [171]:
pd.merge(lncrnas[['gene_id_stable', 'lncrna_category']], gencode_ids, on='gene_id_stable', how='right').to_csv('/home/klawren/oak/gtex/data/other_references/gencode/gencode_id_matching_lncrna_category.txt', sep='\t', index=None)

In [172]:
lncrnas['lncrna_category'].value_counts()

lncrna_category
intergenic                  20740
antisense                   11158
sense                        2485
other                        2191
overlapping_both_strands      531
Name: count, dtype: int64